In [4]:
import torchvision

In [5]:
import torch

In [ ]:
# you can use 
# model = torchvision.models.efficientnet_b0()
# but it has no weights. then if you use this method, you should initilize the weights:

In [ ]:
# In [3] the author stated that the old and new ones are as bellow:
# OLD: Setup the model with pretrained weights and send it to the target device (this was prior to torchvision v0.13)
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD method (with pretrained=True)

# NEW: Setup the model with pretrained weights and send it to the target device (torchvision v0.13+)
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
# model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [ ]:
# So based on the warning I recieve here The parameter 
# 'pretrained=True' is deprecated since 0.13 and will be removed in 0.15

In [19]:
# First I used this method, but is not supported in future. So let's learn and use the other suggested one
# model = torchvision.models.efficientnet_b0(pretrained=True)

In [21]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.efficientnet_b0(weights=weights).to('cpu')

In [22]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [10]:
# you can also use torch Hub for loading models. Researcher put their model on this Hub for others 
# to use.

In [25]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
img = image.imread("data/train/dogs/dog_32.jpg")
import torchvision.transforms.functional as F
img = F.to_pil_image(img)

In [34]:
tranformT = torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
#         transforms.CenterCrop(100),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])

In [35]:
# img.shape

In [38]:
# img

In [41]:
trans_sqzed_img = tranformT(img).unsqueeze(0)

In [44]:
model.eval()
with torch.no_grad():
     print(model(trans_sqzed_img).shape)

torch.Size([1, 1000])


Our efficientnet_b0 comes in three main parts:

   - **features** : A collection of convolutional layers and other various activation layers to learn a base representation of vision data (this base representation/collection of layers is often referred to as features or feature extractor, "the base layers of the model learn the different features of images").
   - **avgpool** : Takes the average of the output of the features layer(s) and turns it into a feature vector.
   - **classifier** : Turns the feature vector into a vector with the same dimensionality as the number of required output classes (since efficientnet_b0 is pretrained on ImageNet and because ImageNet has 1000 classes, out_features=1000 is the default).


In [54]:
# one of the very helpful ways to see how a model works is using torchinfo.summary:
# you only need to give it the size of an input and see how it works through the conv and other layers:

In [70]:
from torchinfo import summary

batch_size = 2
summary(model, input_size=(batch_size, 3, 4, 4), verbose=0)

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.


Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [2, 1000]                 --
├─Sequential: 1-1                                       [2, 1280, 1, 1]           --
│    └─Conv2dNormActivation: 2-1                        [2, 32, 2, 2]             --
│    │    └─Conv2d: 3-1                                 [2, 32, 2, 2]             864
│    │    └─BatchNorm2d: 3-2                            [2, 32, 2, 2]             64
│    │    └─SiLU: 3-3                                   [2, 32, 2, 2]             --
│    └─Sequential: 2-2                                  [2, 16, 2, 2]             --
│    │    └─MBConv: 3-4                                 [2, 16, 2, 2]             1,448
│    └─Sequential: 2-3                                  [2, 24, 1, 1]             --
│    │    └─MBConv: 3-5                                 [2, 24, 1, 1]             6,004
│    │    └─MBConv: 3-6                              

In [74]:
# a good practice is bellow:(trainable and inputsize make is easy for reading and understanding)

In [75]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 4, 4), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 4, 4]        [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 4, 4]        [32, 1280, 1, 1]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 4, 4]        [32, 32, 2, 2]       --                   True
│    │    └─Conv2d (0)                                       [32, 3, 4, 4]        [32, 32, 2, 2]       864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 2, 2]       [32, 32, 2, 2]       64                   True
│    │    └─SiLU (2)                                         [32, 32, 2, 2]       [32, 32, 2, 2]       --                   --
│    └─Sequential (1)                                        [32, 32, 2, 2]       [32, 16, 2, 

In [ ]:
# see and read more about torchinfo.summary options at [4].

In [68]:
# there are two important note to catch (not only used in transferlearning, but in general):
# The size mimatches in here can be tackled from two perspective:
# 1) if it is before training, transform.resize can help to have homogenous shapes
# 2) if it's in training, torch.nn.AdaptiveAvgPool2d(output_size) can handle size mismatches

# Main takeaways


In [76]:
# tranfer learning is a great technique to use others' knowledge and models.
# torchinfo.summary make it easier for reading and understanding the mdoels.
# it's better if we use a newer way of loading models that explained on top of this notebook.

# Resourses

In [ ]:
# [3] https://www.learnpytorch.io/06_pytorch_transfer_learning/
# [4] https://github.com/TylerYep/torchinfo#documentation